In [13]:
# You may need to install the « cryptography » module

import os
import time
import codecs
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.twofactor.hotp import HOTP
from cryptography.hazmat.primitives.hashes import SHA1


In [14]:
def brute_force(value_to_find, key_bits_length, hotp_length, time_limit_sec):
    print('HOTP value to find: ', value_to_find)

    solution = None
    tries_counter = 0
    t_start = time.time()
    # Brute force time limit
    t_end = t_start + time_limit_sec

    while time.time() < t_end:
        # Random key generation
        _key_bits = os.urandom(key_bits_length)
        _HTOP = HOTP(_key_bits, hotp_length, SHA1(), backend=default_backend())
        _hotp_value = _HTOP.generate(0)

        tries_counter += 1

        # Tests this key
        try:
            # It works !
            hotp.verify(_hotp_value, 0)
            solution = _hotp_value
            t_end = time.time()
            break
        except:
            # Not this time...
            pass
        
    # Prints some computation information
    print('Number of tries: ', tries_counter)
    print('Time elapsed: %.2f [s]' % (t_end - t_start))

    if solution:
        print('A solution has been found: ', solution)
        #print(codecs.encode(key_bits, 'hex'))
        #print(codecs.encode(solution, 'hex'))
    else:
        print('No solution found...')

In [ ]:
# The key should be randomly generated and be at least 128 bits ; it is
# recommended to be 160 bits length (20 * 8). This key MUST be kept secret.
KEY_BITS_LENGTH = 20
key_bits = os.urandom(KEY_BITS_LENGTH)
# The length controls the length of the generated one time password
# and must be between 6 and 8.
hotp_length = 6
hotp = HOTP(key_bits, hotp_length, SHA1(), backend=default_backend())
hotp_value = hotp.generate(0)
# Raises a InvalidToken if the hotp_value cannot be verified
hotp.verify(hotp_value, 0)

brute_force(hotp_value, KEY_BITS_LENGTH, hotp_length, 60)

HOTP value to find:  b'418386'


hotp_value to find:  b'675443'
No solution found...
Number of tries:  979880
Time elapsed: 60.00 [s]
